In [1]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pathlib
import PIL
import PIL.Image

In [2]:
data_dir = pathlib.Path('./data')
data = list(data_dir.glob('*/*.*'))

In [7]:
batch_size = 17
img_height = 227
img_width = 227



train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 555 files belonging to 2 classes.
Using 444 files for training.


In [8]:
print(train_ds)

<BatchDataset shapes: ((None, 227, 227, 3), (None,)), types: (tf.float32, tf.int32)>


In [15]:
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False,
        pooling='max',
        input_shape=(img_height, img_width, 3),
        weights='imagenet')

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(512, 'relu'),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(2, 'softmax')
])
    
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
dense_9 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 514       
Total params: 24,745,730
Trainable params: 24,700,290
Non-trainable params: 45,440
_________________________________________________________________


In [16]:
snapshot_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="./snapshots",
    verbose=1)

In [18]:
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

history = model.fit(
    train_ds,
    steps_per_epoch=2, #use way more steps here: number of samples / batch size
    epochs=30 #use way more or use EarlyStopping callback
)

Epoch 1/30
2/2 [==============================] - 8s 1s/step - loss: 91.9401 - sparse_categorical_accuracy: 0.6471
Epoch 2/30
2/2 [==============================] - 3s 1s/step - loss: 91.9715 - sparse_categorical_accuracy: 0.5000
Epoch 3/30
2/2 [==============================] - 3s 2s/step - loss: 53.5665 - sparse_categorical_accuracy: 0.6471
Epoch 4/30
2/2 [==============================] - 3s 1s/step - loss: 15.9783 - sparse_categorical_accuracy: 0.7941
Epoch 5/30
2/2 [==============================] - 3s 1s/step - loss: 6.7833 - sparse_categorical_accuracy: 0.7941
Epoch 6/30
2/2 [==============================] - 3s 1s/step - loss: 18.6340 - sparse_categorical_accuracy: 0.6176
Epoch 7/30
2/2 [==============================] - 3s 2s/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000
Epoch 8/30
2/2 [==============================] - 3s 2s/step - loss: 5.7679 - sparse_categorical_accuracy: 0.9412
Epoch 9/30
2/2 [==============================] - 3s 2s/step - loss: 5.8252 - s